# Model builder
## Life Cycle Assessment of ``{Operational Energy Moerschberg for 1 year: 1}``
###[CONTEXT]
The model relies on datapackages to ensure replicability of the calculation.


In [ ]:
# "combinatorial"
# "sequential"

In [146]:
###Uncomment to restore everything

# bd.projects.delete_project('ei38-teaching-25', delete_dir=True)

'default'

In [364]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import bw_processing as bwp
import numpy as np
from pathlib import Path


if 'ei38-teaching-25' not in bd.projects:
    bi.restore_project_directory("/srv/data/projects/ecoinvent38-25.tar.gz")
# bd.projects.delete_project("moeschpond1", delete_dir=True)
# if 'moeschpond1' not in bd.projects:    
#     backup="brightway2-project-moeschpond1-backup.27-October-2022-09-50AM.tar.gz"
#     premise_path  = "/srv/scratch/u.2/" + backup
#     bi.restore_project_directory(premise_path)
# bd.projects.set_current('moeschpond1')
bd.projects.set_current('ei38-teaching-25')


Restoring project backup archive - this could take a few minutes...


In [363]:
bd.projects.delete_project("ei38-teaching-25")

'default'

In [365]:
bd.databases

Databases dictionary with 2 object(s):
	biosphere3
	ei 3.8 cutoff

In [366]:
# if "foreground" in bd.databases:
#     del bd.databases["foreground"]
    
foreground_importer = bi.ExcelImporter("./data/inputs/lci_moeschberg.xlsx")
# for act in foreground_importer.data:
#     # print(act["name"])
#     for exc in act["exchanges"]:
#         if "classifications" in exc:
#             exc.pop("classifications")
#         if "properties" in exc:
#             exc.pop("properties")   
foreground_importer.apply_strategies()
foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
foreground_importer.statistics()
foreground_importer.write_database()

Extracted 1 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 7.64 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
5 datasets
18 exchanges
0 unlinked exchanges
  
Title: Writing activities to SQLite3 da

In [323]:
# foreground_importer = bi.ExcelImporter("./data/inputs/layered oxide cathode type_HC anode SIB .xlsx")
# # for act in foreground_importer.data:
# #     # print(act["name"])
# #     for exc in act["exchanges"]:
# #         if "classifications" in exc:
# #             exc.pop("classifications")
# #         if "properties" in exc:
# #             exc.pop("properties")   
# foreground_importer.apply_strategies()
# foreground_importer.match_database("biosphere3", fields=("name", "unit", "categories"))
# foreground_importer.match_database("ei 3.8 cutoff", fields=("name", "unit", "location"))
# foreground_importer.statistics()
# foreground_importer.write_database()

Extracted 1 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields


KeyboardInterrupt: 

In [367]:
fu = {bd.get_activity(database="energy_moeschberg", name = "energy demand, operational, Hotel Moeschberg"):1}

In [355]:
bd.get_activity(id=18352)

'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, CH, None)

In [356]:
# ## Creating datapackage for each method
# method_hint = "IPCC"
# data_objs_methods = []
# methods = [method for method in bd.methods if method_hint in method[0]]
# for method in methods:
#     _, objs, __ = bd.prepare_lca_inputs(method=method)
#     data_objs_methods += objs

In [368]:
### it returns functional_unit, data_objects, and a mapping dictionary
fu, objects, remapping_dicts = bd.prepare_lca_inputs(fu, method=('IPCC 2013', 'climate change', 'GWP 100a'))


In [358]:
for obj in objects:
    print(obj.metadata["name"])

biosphere3
ei_3.8_cutoff
energy_moeschberg
ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf.zip


In [369]:
data, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.data")
indices, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.indices")
flip, _ = objects[2].get_resource("energy_moeschberg_technosphere_matrix.flip")
unique_indices = set([a for b in indices for a in b])
mapping_act=dict.fromkeys(int(i) for i in unique_indices) #json accepts only int not int32
for i in unique_indices:
    mapping_act[i]=bd.get_activity(i)['name']
rows = [tup[0] for tup in indices]
cols = [tup[1] for tup in indices]

In [361]:
indices

array([( 6599, 24005), ( 6599, 24006), ( 8302, 24007), (10319, 24006),
       (18352, 24004), (18352, 24005), (18352, 24006), (20079, 24006),
       (23758, 24007), (24004, 24004), (24004, 24008), (24005, 24004),
       (24005, 24005), (24006, 24005), (24006, 24006), (24007, 24007),
       (24007, 24008), (24008, 24008)],
      dtype=[('row', '<i4'), ('col', '<i4')])

In [370]:
import pandas as pd
matrix_raw = pd.DataFrame({"row":rows, 
                           "col":cols, 
                           "from":[mapping_act[idx] for idx in rows], 
                           "to":[mapping_act[idx] for idx in cols],
                           "data":data
                          })
matrix = matrix_raw.pivot(index='row',columns='col', values='data').fillna(0)
# mapping
matrix_raw

,row,col,from,to,data
0,6599,23994,"electricity production, hydro, run-of-river","electricity supply, Energie Grosshoechstetten ...",0.907000
1,6599,23995,"electricity production, hydro, run-of-river","electricity, subsidised, 2021",0.320000
2,8302,23996,"operation, solar collector system, Cu flat pla...","heat supply, Hotel Moeschberg, 2021",0.004668
3,10319,23995,"electricity production, wind, 1-3MW turbine, o...","electricity, subsidised, 2021",0.020000
4,18352,23993,"electricity production, photovoltaic, 3kWp sla...","electricity supply, Hotel Moeschberg, 2021",0.440600
5,18352,23994,"electricity production, photovoltaic, 3kWp sla...","electricity supply, Energie Grosshoechstetten ...",0.026000
6,18352,23995,"electricity production, photovoltaic, 3kWp sla...","electricity, subsidised, 2021",0.460000
7,20079,23995,"heat and power co-generation, wood chips, 6667...","electricity, subsidised, 2021",0.200000
8,23758,23996,"heat production, wood pellet, at furnace 25kW,...","heat supply, Hotel Moeschberg, 2021",0.995300
9,23993,23993,"electricity supply, Hotel Moeschberg, 2021","electricity supply, Hotel Moeschberg, 2021",1.000000


In [371]:
objects[2].metadata["name"]
# dp_dict = dict(zip(indices.astype("object"),list(zip(data,flip))))

dp_dict = dict(
        zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
                )
dp_dict


{(6599, 23994): [0.907, True],
 (6599, 23995): [0.32, True],
 (8302, 23996): [0.004668, True],
 (10319, 23995): [0.02, True],
 (18352, 23993): [0.4406, True],
 (18352, 23994): [0.026, True],
 (18352, 23995): [0.46, True],
 (20079, 23995): [0.2, True],
 (23758, 23996): [0.9953, True],
 (23993, 23993): [1.0, False],
 (23993, 23997): [32179.0, True],
 (23994, 23993): [0.5594, True],
 (23994, 23994): [1.0, False],
 (23995, 23994): [0.067, True],
 (23995, 23995): [1.0, False],
 (23996, 23996): [1.0, False],
 (23996, 23997): [655489.0, True],
 (23997, 23997): [1.0, False]}

In [305]:
new_data = np.array([val[0] for val in dp_dict.items()])

In [379]:
new_arrays = {
    (18352,23993): [0.4406,1], #PV becomes the unique elec source
    (23994,23993): [0.5594,0],
    (23758,23996): [0.995300,0], #wood pellets are replaced by solar thermal
    (8302,23996): [0.004668,1],
            }
def modify_w_arrays(data_object,new_arrays):
    n = 1
    for k,v in new_arrays.items():
        n = len(v)        

    data, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.data")
    indices, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.indices")
    flip, _ = data_object.get_resource(f"{data_object.metadata['name']}_technosphere_matrix.flip")
    dp_dict = dict(
        zip(indices.astype("object"),[[tup[0], tup[1]] for tup in zip(data,flip)])
                        )
    for key, value in new_arrays.items():
        dp_dict[key][0] = value
    
    new_foreground = bwp.create_datapackage(
    fs = bwp.generic_zipfile_filesystem(dirpath=Path("./data/inputs"), filename=f"{data_object.metadata['name']}.zip", write=True),
    # combinatorial=True,
    sequential=True,
    )

    for row_col, data_flip in dp_dict.items():
        if not isinstance(data_flip[0], np.ndarray):
            new_array = np.full((n,), fill_value = data_flip[0])
            dp_dict[row_col][0] = new_array
            
    new_data = np.array([val[0] for val in dp_dict.items()])
                   
    new_foreground.add_persistent_array(
    matrix="technosphere_matrix",
    data_array=new_data,
    indices_array=indices.astype(bwp.INDICES_DTYPE),
    flip_array=flip,
    name=data_object.metadata['name'],
)

    # return dp_dict
    return new_foreground

In [380]:
aa = modify_w_arrays(objects[2], new_arrays)

In [385]:
lca_b = bc.LCA(
    demand=fu,
    data_objs=objects + [aa],
    use_distributions=False,
    use_arrays=True,
#     seed_override=42,  # Seed should not be used
)
lca_b.lci()
lca_b.lcia()
lca_b.keep_first_iteration()
iterations = 10
scores_b = [lca_b.score for _ in zip(range(iterations), lca_b)]
scores_b

[0.2469761531596589,
 0.33116044116494214,
 0.2469761531596589,
 0.33116044116494214,
 0.2469761531596589,
 0.33116044116494214,
 0.2469761531596589,
 0.33116044116494214,
 0.2469761531596589,
 0.33116044116494214]